In [6]:
import pandas as pd
path = r"D:\Online_Retail.csv"
##路径
df = pd.read_csv(path, encoding='latin-1')
##注意用, encoding='latin-1'，不然会报错
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [7]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


由上面可以知道数据集大小memory usage: 33.1+ MB，数据类型两个float64，1个int64,5个object，总共8列。
参考官网的信息可以知道
'InvoiceNo': '发票号码 - 6位整数，唯一标识每笔交易。以字母"C"开头的表示取消订单',
    'StockCode': '产品代码 - 5位整数，唯一标识每个不同的产品',
    'Description': '产品名称描述',
    'Quantity': '购买数量 - 每笔交易中每个产品的购买数量',
    'InvoiceDate': '发票日期和时间 - 生成每笔交易的日期和时间',
    'UnitPrice': '单价 - 每个产品的单位价格（英镑）',
    'CustomerID': '客户ID - 5位整数，唯一标识每个客户',
    'Country': '国家名称 - 客户所在国家的名称'


In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


上面的对数值型数据进行描述统计，分别是两个float64，1个int64；购买数量，购买金额，用户ID
简单观测数据，发现数量最大的一笔交易应该发生了退款，因为最大和最下的数量刚刚好是一样的正负数

数量的均值 为 9.55，说明平均每笔交易购买了约 9.5 个商品且标准差很大（218） ，说明数量波动非常大。

金额平均单价为 4.61 英镑 （单位是英镑），大部分商品价格较低。最贵的金额是38970，标准差高达 96.76 ，说明商品价格差异偏高。
参考价格的四分位曲线，推测价格整体偏低

参考客户ID，数据参考意义不对，但是以看出有大约 40 万条记录包含客户 ID ，而总记录是 54 万，说明有些记录缺失了客户信息

In [9]:
missing_stats = pd.DataFrame({
    '缺失数量': df.isnull().sum(),
    '缺失比例(%)': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_stats

,缺失数量,缺失比例(%)
InvoiceNo,0,0.00
StockCode,0,0.00
Description,1454,0.27
Quantity,0,0.00
InvoiceDate,0,0.00
UnitPrice,0,0.00
CustomerID,135080,24.93
Country,0,0.00


产品名称描述缺失了1454个，包括之前的客户ID 135080个

In [10]:
# 检查异常的发票号码（以'C'开头表示取消的订单）
cancelled_orders = df[df['InvoiceNo'].astype(str).str.startswith('C', na=False)]
print(f"取消订单数量: {len(cancelled_orders)}")
print(f"取消订单比例: {(len(cancelled_orders) / len(df) * 100):.2f}%")

# 查看取消订单示例
if len(cancelled_orders) > 0:
    print("\n取消订单示例:")
    print(cancelled_orders.head())

取消订单数量: 9288
取消订单比例: 1.71%

取消订单示例:
    InvoiceNo StockCode                       Description  Quantity  \
141   C536379         D                          Discount        -1   
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

       InvoiceDate  UnitPrice  CustomerID         Country  
141   12/1/10 9:41      27.50     14527.0  United Kingdom  
154   12/1/10 9:49       4.65     15311.0  United Kingdom  
235  12/1/10 10:24       1.65     17548.0  United Kingdom  
236  12/1/10 10:24       0.29     17548.0  United Kingdom  
237  12/1/10 10:24       0.29     17548.0  United Kingdom  


参考上面的结果可以知道实际取消得到订单数量偏低，仅为1.7%左右

In [11]:
# 检查负数购买数量
negative_quantity = df[df['Quantity'] < 0]
print(f"负数购买数量记录: {len(negative_quantity)}")
print(f"负数购买数量比例: {(len(negative_quantity) / len(df) * 100):.2f}%")

if len(negative_quantity) > 0:
    print("\n负数购买数量统计:")
    print(negative_quantity['Quantity'].describe())
    print("\n负数购买数量示例:")
    print(negative_quantity[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice']].head())

负数购买数量记录: 10624
负数购买数量比例: 1.96%

负数购买数量统计:
count    10624.000000
mean       -45.607210
std       1092.214216
min     -80995.000000
25%        -10.000000
50%         -2.000000
75%         -1.000000
max         -1.000000
Name: Quantity, dtype: float64

负数购买数量示例:
    InvoiceNo StockCode                       Description  Quantity  UnitPrice
141   C536379         D                          Discount        -1      27.50
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1       4.65
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12       1.65
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24       0.29
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24       0.29


负数购买数量比例: 1.96%，同样偏低，注意这里是退货回来

In [12]:
# 检查异常价格
print("价格异常检查:")
print(f"零价格记录数量: {len(df[df['UnitPrice'] == 0])}")
print(f"负价格记录数量: {len(df[df['UnitPrice'] < 0])}")
print(f"异常高价格记录数量: {len(df[df['UnitPrice'] > 1000])}")

# 价格分布统计
print(f"\n价格统计:")
print(df['UnitPrice'].describe())

# 查看异常价格记录
extreme_prices = df[(df['UnitPrice'] <= 0) | (df['UnitPrice'] > 1000)]
if len(extreme_prices) > 0:
    print(f"\n异常价格记录示例:")
    print(extreme_prices[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice']].head())

价格异常检查:
零价格记录数量: 2515
负价格记录数量: 2
异常高价格记录数量: 120

价格统计:
count    541909.000000
mean          4.611114
std          96.759853
min      -11062.060000
25%           1.250000
50%           2.080000
75%           4.130000
max       38970.000000
Name: UnitPrice, dtype: float64

异常价格记录示例:
     InvoiceNo StockCode Description  Quantity  UnitPrice
622     536414     22139         NaN        56        0.0
1970    536545     21134         NaN         1        0.0
1971    536546     22145         NaN         1        0.0
1972    536547     37509         NaN         1        0.0
1987    536549    85226A         NaN         1        0.0


价格异常检查:
零价格记录数量: 2515
负价格记录数量: 2
异常高价格记录数量: 120

注意，这是一个中位数为2.08英镑，平均值为4.61 英镑，价格特别高的记录数量为120条，这可能表明存在异常高价商品。

In [13]:
# 检查无效的客户ID
invalid_customer_id = df[df['CustomerID'].isnull()]
print(f"缺失客户ID记录数量: {len(invalid_customer_id)}")
print(f"缺失客户ID比例: {(len(invalid_customer_id) / len(df) * 100):.2f}%")

# 客户ID统计
valid_customers = df[df['CustomerID'].notnull()]
print(f"有效客户数量: {valid_customers['CustomerID'].nunique()}")
print(f"客户ID范围: {valid_customers['CustomerID'].min()} - {valid_customers['CustomerID'].max()}")

缺失客户ID记录数量: 135080
缺失客户ID比例: 24.93%
有效客户数量: 4372
客户ID范围: 12346.0 - 18287.0


缺失客户ID记录数量: 135080
缺失客户ID比例: 24.93%
有效客户数量: 4372
客户ID范围: 12346.0 - 18287.0
参考这个信息可以发现，交易单数达50万的数据，客户仅仅为不到5000个，这意味着大部分的交易都是由少数客户完成的。同时，约有24.93%的记录缺少了客户ID，这对于数据分析来说是一个需要注意的问题，因为缺失数据可能会影响分析结果的准确性或完整性。

In [16]:
# 转换日期格式，注意参考数据中的日期格式（格式为'%Y/%m/%d %H:%M:%S'）
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%Y/%m/%d %H:%M:%S')

# 检查日期范围
print(f"数据时间范围: {df['InvoiceDate'].min()} 到 {df['InvoiceDate'].max()}")
print(f"数据跨度: {(df['InvoiceDate'].max() - df['InvoiceDate'].min()).days} 天")

# 提取日期特征
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['Hour'] = df['InvoiceDate'].dt.hour

print(f"\n年份分布:")
print(df['Year'].value_counts().sort_index())

数据时间范围: 2010-12-01 08:26:00 到 2011-12-09 12:50:00
数据跨度: 373 天

年份分布:
Year
2010     42481
2011    499428
Name: count, dtype: int64


参考数据集的数量接近50万条，时间却不到一年半，说明数据集的生成速度很快，且客户数少，复购率高，说明用户粘性很高。

In [17]:
# 创建促销期标识
# 基于常见的促销时期：黑色星期五、圣诞节期间、新年期间等

def identify_promotion_period(date):
    """
    识别促销期
    基于常见促销时间段:
    - 11月（黑色星期五/网络星期一）
    - 12月（圣诞节促销）
    - 1月（新年促销）
    """
    month = date.month
    day = date.day
    
    # 黑色星期五周期（11月第四个星期）
    if month == 11 and day >= 22:
        return 'Black_Friday'
    # 圣诞节促销期
    elif month == 12:
        return 'Christmas'
    # 新年促销期
    elif month == 1:
        return 'New_Year'
    # 夏季促销期（7-8月）
    elif month in [7, 8]:
        return 'Summer_Sale'
    else:
        return 'Regular'

# 应用促销期标识
df['PromotionPeriod'] = df['InvoiceDate'].apply(identify_promotion_period)

# 查看促销期分布
print("促销期分布:")
promotion_stats = df['PromotionPeriod'].value_counts()
print(promotion_stats)
print(f"\n促销期比例:")
print((promotion_stats / len(df) * 100).round(2))

促销期分布:
PromotionPeriod
Regular         335837
Summer_Sale      74802
Christmas        68006
New_Year         35147
Black_Friday     28117
Name: count, dtype: int64

促销期比例:
PromotionPeriod
Regular         61.97
Summer_Sale     13.80
Christmas       12.55
New_Year         6.49
Black_Friday     5.19
Name: count, dtype: float64


Regular（非促销期） ：这是占比最大的类别，表示大多数交易发生在没有特别促销活动的时间段。
Summer_Sale（夏季促销） ：第二大的促销期，说明在每年 7、8 月份有较明显的促销活动或销售高峰。
Christmas（圣诞节促销） ：紧随其后，符合节日消费高峰的特点。
New_Year（新年促销） 和 Black_Friday（黑色星期五） ：这两个促销期相对较小，但仍具有一定的销售影响力。

超过 60% 的交易发生在非促销期 ，说明促销活动虽然存在，但对整体交易量的影响相对有限。
夏季促销 和 圣诞节促销 是最显著的两个促销节点，合计占比超过 26% ，是值得关注的重点时段。
黑色星期五 和 新年促销 虽然也有一定影响，但占比相对较低，可能是因为时间较短、宣传力度不够或地域限制等因素。

In [39]:
# 创建清洗后的数据集(新建一个副本df_cleaned)
df_cleaned = df.copy()

In [40]:
# 1. 删除缺失客户ID的记录
df_cleaned = df_cleaned[df_cleaned['CustomerID'].notnull()]  #去除缺失的客户ID
print(f"删除缺失客户ID的记录: {len(df_cleaned)}")
# 2. 删除重复记录
df_cleaned = df_cleaned.drop_duplicates()
print(f"删除重复记录后: {len(df_cleaned)}")

删除缺失客户ID的记录: 406829
删除重复记录后: 401604


观测后发现前后从50万的数据集降到40万数据，推测数据治理不好，需要加强信息建设，提高数据质量。

In [41]:
# 3. 处理异常值
# 删除异常的价格和数量（考虑数量和价格的均值	9.552250和4.611114	）
df_cleaned = df_cleaned[
    (df_cleaned['UnitPrice'] > 0) & 
    (df_cleaned['UnitPrice'] < 1000) &  # 设单价不超过1000英镑
    (df_cleaned['Quantity'] > 0) &
    (df_cleaned['Quantity'] < 1000)     # 设单次购买不超过1000件
]
print(f"删除异常价格和数量后: {len(df_cleaned)}")

删除异常价格和数量后: 392557


In [42]:
# 4. 删除取消的订单（分析退货）
df_cleaned = df_cleaned[~df_cleaned['InvoiceNo'].astype(str).str.startswith('C', na=False)]
print(f"删除取消订单后: {len(df_cleaned)}")

print(f"\n最终清洗后数据行数: {len(df_cleaned)}")
print(f"数据保留比例: {(len(df_cleaned) / len(df) * 100):.2f}%")

删除取消订单后: 392557

最终清洗后数据行数: 392557
数据保留比例: 72.44%


右上可得，数据大部分异常出现为缺失的顾客ID信息。

复购 = 同一个客户在不同日期 下的订单（即忽略同一天多次下单） 

In [43]:
# 1. 提取每个客户的购买日期（只保留日期部分，去掉时间）
df_cleaned['InvoiceDateOnly'] = df_cleaned['InvoiceDate'].dt.date

# 2. 按 CustomerID 分组，对每个客户按 InvoiceDateOnly 排序并编号，第一次为1，第二次为2，依此类推
df_cleaned['PurchaseOrder'] = df_cleaned.groupby('CustomerID')['InvoiceDateOnly'].rank(method='dense')

# 3. 如果 PurchaseOrder > 1，则表示是复购订单（不是第一次购买）
df_cleaned['IsRepurchase'] = df_cleaned['PurchaseOrder'] > 1

# ✅ 把布尔值转换为 0 和 1,方便EXcel处理，True→1, False→0  
df_cleaned['IsRepurchase'] = df_cleaned['IsRepurchase'].astype(int)   

In [44]:
# 4. 导出用于Excel透视表的数据(“促销期”、“年”、“月”、“是否回购”、“发票号”、“客户ID”)
##备注InvoiceNo，中文是“发票编号”或“订单编号”，在你的数据中代表每一次交易的唯一标识。
output_columns = ['PromotionPeriod', 'Year', 'Month', 'IsRepurchase', 'InvoiceNo', 'CustomerID']
df_for_excel = df_cleaned[output_columns]

# 5. 导出到Excel文件
df_for_excel.to_excel("Excel_Analysis.xlsx", index=False)